<a href="https://colab.research.google.com/github/Gajendra-theDataEngineer004627/BigDataAssignments/blob/Assignment11/Assignment11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [494 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [984 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [990 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [909 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
!pip install -q findspark
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, DoubleType

In [8]:
spark = SparkSession.builder \
    .appName("Assignemnt11") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.2") \
    .master("local[*]") \
    .getOrCreate()


In [9]:
window_schema = StructType([
    StructField("country", StringType()),
    StructField("weeknum",IntegerType()),
    StructField("numinvoices",IntegerType()),
    StructField("totalquantity", IntegerType()),
    StructField("invoicevalue",DoubleType())
])


In [10]:
# function to doanload data files from the google drive
import gdown
def downloadfiles_fromFolder() :
    url = "https://drive.google.com/drive/folders/1Hf8PijpBSNyDjwr-YgozGDRL8UqMDq3D?usp=sharing"   # can also be used input()
    if url.split("/")[-1]== "?usp=sharing":
       url = url.replace("?usp=sharing","")
    gdown.download_folder(url, output="/content")

downloadfiles_fromFolder()

Retrieving folder list


Retrieving folder 1PzvW3sL9CcaFgkFqywCKI8PUiYVUE4kt DataSets
Processing file 1Daxk18O6UzEiCQpsA8j_3zaWVuTjGjxl windowdata.csv
Retrieving folder 1mMKOGZGhZV4trkCrDX352AiaKz4sVeA0 Output_of_Assignment
Retrieving folder 1O-xLP9OssEXWhG3b1EOd-CwCDhrwdkMW Assignment11
Retrieving folder 1Omu_t1Zt3lqa8Vx7F-7naLtUNhv_lAuC test
Processing file 16D6BP-5HbStzW2nTaVMxz1bpX4ye9slY week11_assignment.pdf
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1Daxk18O6UzEiCQpsA8j_3zaWVuTjGjxl
To: /content/DataSets/windowdata.csv
100%|██████████| 1.30k/1.30k [00:00<00:00, 5.06MB/s]
Downloading...
From: https://drive.google.com/uc?id=16D6BP-5HbStzW2nTaVMxz1bpX4ye9slY
To: /content/week11_assignment.pdf
100%|██████████| 657k/657k [00:00<00:00, 80.3MB/s]
Download completed


In [ ]:
"""   ---Its a function to save files as csv to the google drive---------
import os
import pandas as pd

def spark_dataframes_to_csv(dataframes, drive_path):
    # Mount Google Drive if not already mounted
    if not os.path.isdir('/content/gdrive'):
        from google.colab import drive
        drive.mount('/content/gdrive')

    # Create directory if it doesn't exist
    directory_path = '/content/gdrive/MyDrive/' + drive_path
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    # Convert Spark DataFrames to Pandas DataFrames and save as CSV
    for i, dataframe in enumerate(dataframes, start=1):
        pandas_df = dataframe.toPandas()
        file_path = f'{directory_path}/df{i}.csv'
        pandas_df.to_csv(file_path, index=False)
        print(f"DataFrame {i} saved as {file_path}")

"""

In [ ]:
path1 = "/content/DataSets/windowdata.csv"

# -----Reading the data using standard spark reader Api
window_dataframe = spark.read.format("csv").schema(window_schema).option("path",path1).load()
window_dataframe.show()


In [13]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# using write api for saving data in parquet format
target_path1 = "/content/drive/MyDrive/Assignment11/"
# Saving in the parquet format
window_dataframe.write.partitionBy("country","weeknum").mode("overwrite").parquet(target_path1)
print("Parquet data saved to Google Drive successfully!")

Parquet data saved to Google Drive successfully!


In [21]:
import shutil

# Specify the local target path for Avro files
local_target_path_avro = "/content/AvroData"

target_path2 = "/content/drive/MyDrive/Assignment11_testAvro"
# saving it in avro format
window_dataframe.write.partitionBy("country").format("avro").mode("overwrite").save(target_path2)
#shutil.move(local_target_path_avro, target_path2)
print("Avro data saved to Google Drive successfully!")

print("Finally Successfully solved problem 1....!!!")

Avro data saved to Google Drive successfully!
Finally Successfully solved problem 1....!!!


In [22]:
# Unmount Google Drive to release the connection
# after this all data will be saved to drive
drive.flush_and_unmount()